# Model Creation

                                                                                        Code Written By : Dasari Mohana
                                                                                        
We have done EDA and Feature Engineering on raw dataset and got final dataset with neccessary changes(removing duplicates, scaling the data, Multicolinearity, replacing null values, Removing outliers)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import xgboost

from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso, SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

%matplotlib inline
pd.set_option('display.max_columns',None)
warnings.simplefilter(action='ignore')

In [2]:
df = pd.read_csv('final_zillow_dataset.csv')
df.shape

(70260, 21)

In [3]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,finishedsquarefeet12,fips,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,taxamount,logerror,yeardifference,propertyzoningdesc_labels
0,10726315,3.0,3.0,4.0,2445.0,6037.0,2.0,34.184300,-118.657000,63878.0,266.0,60.371352,12447.0,3101.0,96389.0,0.0,1.0,7170.22,0.0383,33.0,627
1,10727091,2.0,3.0,7.0,1160.0,6037.0,2.0,34.188121,-118.646361,7688.0,261.0,60.371352,12447.0,3101.0,96342.0,0.0,1.0,5421.96,0.0030,55.0,638
2,10730788,2.0,4.0,7.0,1570.0,6037.0,2.0,34.188446,-118.616724,11308.0,261.0,60.371351,12447.0,3101.0,96342.0,0.0,1.0,5097.78,-0.0020,56.0,638
3,10735394,3.0,4.0,4.0,2863.0,6037.0,2.0,34.171439,-118.646883,16376.0,261.0,60.371373,12447.0,3101.0,96389.0,0.0,1.0,7475.21,0.0129,52.0,632
4,10743512,3.0,2.0,4.0,1394.0,6037.0,2.0,34.152890,-118.791494,77543.0,269.0,60.378003,34278.0,3101.0,96385.0,0.0,1.0,5550.36,0.0630,17.0,769


### Splitting the dataset into Train-Test sets

In [4]:
X = df.drop('logerror', axis=1)
y = df['logerror']

new_df = df.copy()

print(X.shape, y.shape)

((70260, 20), (70260,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 23)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((52695, 20), (52695,), (17565, 20), (17565,))

### Feature Scaling

In [6]:
train_vars = [var for var in X_train.columns if var not in ['parcelid', 'logerror']]
len(train_vars)

19

In [7]:
scaler = StandardScaler()

scaler.fit(X_train[train_vars]) 

X_train[train_vars] = scaler.transform(X_train[train_vars])

X_test[train_vars] = scaler.transform(X_test[train_vars])

In [8]:
X_train.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,finishedsquarefeet12,fips,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,taxamount,yeardifference,propertyzoningdesc_labels
15602,10934030,1.039614,1.077319,-1.450606,1.368148,-0.672408,-0.526789,0.533872,-0.840819,-0.282457,-0.50568,-0.703341,-0.813301,0.672408,-1.208086,-0.439343,-0.081247,-0.954568,1.254891,-0.470974
27292,12116223,-1.441419,0.059583,0.629159,0.535918,-0.672408,-0.526789,0.674431,-0.036819,-0.267228,-0.50568,-0.671488,0.974128,0.672408,-0.604150,-0.439343,-0.081247,-0.737606,2.552634,1.686830
37134,11533593,-0.200903,0.059583,0.629159,-0.728171,-0.672408,1.921281,0.016094,-1.104821,-0.334003,-0.50568,-0.690187,-0.813301,0.672408,-1.213056,-0.439343,-0.081247,1.636555,0.779052,-0.470974
19660,13078527,-0.200903,1.077319,0.629159,-0.591716,-0.672408,-0.526789,0.122551,0.902323,-0.274704,-0.50568,-0.677385,0.333907,0.672408,-0.032524,-0.439343,-0.081247,-0.229807,0.173439,2.541898
19926,12163237,-0.200903,-0.958153,0.629159,-0.860129,-0.672408,-0.526789,0.764055,-0.126408,-0.311506,-0.50568,-0.671698,0.103886,0.672408,-0.671254,-0.439343,-0.081247,-0.688036,1.903762,0.161445


### Dropping parcelid as it does not serve purpose for model prediction 

In [9]:
X_train_new = X_train.copy()
X_test_new = X_test.copy()

X_train.drop(columns='parcelid', axis=1, inplace=True)
X_test.drop(columns='parcelid', axis=1, inplace=True)

## Linear Regression Model

In [10]:
linear_reg = LinearRegression()

linear_reg.fit(X_train, y_train)

LinearRegression()

In [11]:
linear_reg_pred = linear_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, linear_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, linear_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, linear_reg_pred))))

Mean Absolute Error : 0.05271290201314923

Mean Squared Error : 0.007219676292818219

Root Mean Squared Error : 0.08496867830452713


## Elastic Net Model

In [12]:
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5) # L1 reguralization

elastic_net.fit(X_train, y_train)

ElasticNet(alpha=0.1)

In [13]:
elastic_net_pred = elastic_net.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, elastic_net_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, elastic_net_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, elastic_net_pred))))

Mean Absolute Error : 0.052772037640842616

Mean Squared Error : 0.007263564946868809

Root Mean Squared Error : 0.08522655071554175


## Ridge Regression Model

In [14]:
ridge_reg = Ridge(alpha=1, solver='cholesky')

ridge_reg.fit(X_train, y_train)

Ridge(alpha=1, solver='cholesky')

In [15]:
ridge_reg_pred = ridge_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, ridge_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, ridge_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, ridge_reg_pred))))

Mean Absolute Error : 0.05271298978108195

Mean Squared Error : 0.0072190878626148345

Root Mean Squared Error : 0.08496521560388601


## Lasso Regression Model

In [16]:
lasso_reg = Lasso(alpha=0.1)

lasso_reg.fit(X_train, y_train)

Lasso(alpha=0.1)

In [17]:
lasso_reg_pred = lasso_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, lasso_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, lasso_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, lasso_reg_pred))))

Mean Absolute Error : 0.052772037640842616

Mean Squared Error : 0.007263564946868809

Root Mean Squared Error : 0.08522655071554175


## XG Boost Regression Model

In [18]:
xgb_reg = xgboost.XGBRegressor()

xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [19]:
xgb_reg_pred = xgb_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, xgb_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, xgb_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, xgb_reg_pred))))

Mean Absolute Error : 0.05380005751526278

Mean Squared Error : 0.007353353803794397

Root Mean Squared Error : 0.08575169854757629


## Ada Boost Regression Model

In [20]:
adaboost_reg = AdaBoostRegressor()

adaboost_reg.fit(X_train, y_train)

AdaBoostRegressor()

In [21]:
adaboost_reg_pred = adaboost_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, adaboost_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, adaboost_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, adaboost_reg_pred))))

Mean Absolute Error : 0.05676752363007214

Mean Squared Error : 0.0075389292853823555

Root Mean Squared Error : 0.08682700781083243


## Gradient Boosting Regression Model

In [22]:
gb_reg = GradientBoostingRegressor()

gb_reg.fit(X_train, y_train)

GradientBoostingRegressor()

In [23]:
gb_reg_pred = gb_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, gb_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, gb_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, gb_reg_pred))))

Mean Absolute Error : 0.05256208156829912

Mean Squared Error : 0.007179126861416838

Root Mean Squared Error : 0.08472972832139165


## Decision Tree Regressor

In [24]:
tree_reg = DecisionTreeRegressor(max_depth=5)

tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5)

In [25]:
tree_reg_pred = tree_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, tree_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, tree_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, tree_reg_pred))))

Mean Absolute Error : 0.05291405344779044

Mean Squared Error : 0.007261957829150837

Root Mean Squared Error : 0.08521712169013242


## Random Forest Regression Model

In [26]:
forest_reg = RandomForestRegressor(n_estimators= 500, max_depth=6)

forest_reg.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, n_estimators=500)

In [27]:
forest_reg_pred = forest_reg.predict(X_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, forest_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, forest_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, forest_reg_pred))))

Mean Absolute Error : 0.052493979086337225

Mean Squared Error : 0.0071765998987079645

Root Mean Squared Error : 0.08471481510755934


## Cross Validation & Hyperparameter Optimization for Random Forest

In [28]:
scores = cross_val_score(forest_reg, X_train, y_train, scoring="neg_mean_squared_error", cv = 5)

In [29]:
forest_reg_rmse_scores = np.sqrt(-scores)
forest_reg_rmse_scores

array([0.08603316, 0.08501659, 0.08290164, 0.0832188 , 0.08450688])

In [30]:
param_grid = [
    {'n_estimators': [300, 400, 500], 'max_features': [2, 4, 6]},
    {'bootstrap': [False], 'n_estimators': [300, 400, 500], 'max_features': [2, 4, 6]}]

forest_regressor = RandomForestRegressor()

grid_search = GridSearchCV(forest_regressor, param_grid, scoring='neg_mean_squared_error',return_train_score=True,cv=3)

In [31]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6],
                          'n_estimators': [300, 400, 500]},
                         {'bootstrap': [False], 'max_features': [2, 4, 6],
                          'n_estimators': [300, 400, 500]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [32]:
grid_search.best_params_

{'max_features': 4, 'n_estimators': 400}

In [33]:
grid_search.best_estimator_

RandomForestRegressor(max_features=4, n_estimators=400)

In [35]:
final_predictor = grid_search.best_estimator_
final_predictor.fit(X_train, y_train)
final_pred = final_predictor.predict(X_test)

In [36]:
print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, final_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, final_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, final_pred))))

Mean Absolute Error : 0.0548705301060865

Mean Squared Error : 0.007400295260520292

Root Mean Squared Error : 0.08602496882022273


## Feature importance

In [37]:
feature_importances = grid_search.best_estimator_.feature_importances_

attrs = list(df.select_dtypes(include = ['float64','int64']))

sorted(zip(attrs, feature_importances), reverse=True)

[('unitcnt', 0.14040615200665663),
 ('taxamount', 0.09130069345797628),
 ('roomcnt', 0.0015609032588653922),
 ('regionidzip', 0.008357667884179286),
 ('regionidcounty', 0.059049163834885586),
 ('regionidcity', 0.0009523510814962),
 ('rawcensustractandblock', 0.030844984329222235),
 ('propertylandusetypeid', 0.07946690719058498),
 ('parcelid', 0.023641002820564777),
 ('lotsizesquarefeet', 0.008677886840809414),
 ('longitude', 0.1109836767557538),
 ('logerror', 0.048932900132720544),
 ('latitude', 0.11430406832176654),
 ('heatingorsystemtypeid', 0.11542963001723361),
 ('fips', 0.009994615270457963),
 ('finishedsquarefeet12', 0.0010039810716131004),
 ('buildingqualitytypeid', 0.11671937521605583),
 ('bedroomcnt', 0.008630208122532842),
 ('bathroomcnt', 0.029743832386624945)]

## Saving Predictions

In [38]:
model_pred = pd.DataFrame({'parcelid':X_test_new.parcelid, 'logerror':final_pred})
model_pred.to_csv('model-predictions.csv',index=False)
model_pred.head()

,parcelid,logerror
11056,11923769,0.018472
20399,14196198,0.005623
51034,12408039,0.004737
41560,10931254,0.021955
69896,14665147,-0.034711


# Conclusion

1. After Pre-processing and Feature Engineering the raw dataset we splitted the dataset into train and test sets.

2. Performed Feature scaling on data for better performance.

3. Trained multiple models using different ML regression algorithms on dataset.

4. Appleied Performance metrics such as MAE, MSE, RMSE to find out best prediction model.

5. With the help of GridSearch CV we found out best estimator with least Root mean squred error. 

6. Done prediction on test data and saved predictions into .csv file.